<a href="https://colab.research.google.com/github/jsckim9026/jsckim_01/blob/main/07_Cats_and_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Flatten,Dense,Dropout,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
print(tf.__version__ )


2.7.0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import os
import shutil
#working_dir='dataset'
#colab_default_dir='/content/gdrive/MyDrive/Colab Notebooks/'
original_dir=os.getcwd()
print('current dir = ',os.getcwd())
print('orginal dir = ',str(original_dir))

Mounted at /content/gdrive/
current dir =  /content
orginal dir =  /content


In [ ]:
#!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip


In [ ]:
#import zipfile
#with zipfile.ZipFile('/content/cats_and_dogs_filtered.zip','r') as target_file:
#  target_file.extractall('/content/gdrive/MyDrive/Colab Notebooks/dataset/cats_dogs_filtered/')
  #target_file.extractall('/content/cats_dogs_filtered/')

In [ ]:
IMG_WIDTH=224 
IMG_HEIGHT=224 
base_model=Xception(weights='imagenet',include_top=False, input_shape=(IMG_WIDTH,IMG_HEIGHT,3))
model=Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2,activation='softmax'))
model.summary()

83697664/83683744 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 34        
                                                                 
Total params: 20,894,298
Trainable params: 20,839,770
Non

In [ ]:

path_tmp='/content/gdrive/MyDrive/Colab Notebooks/dataset/cats_dogs_filtered/cats_and_dogs_filtered'
train_dir=os.path.join(path_tmp,'train')
test_dir=os.path.join(path_tmp,'validation')
print(train_dir)

train_data_gen=ImageDataGenerator(rescale=1./255,
                                  rotation_range=10,width_shift_range=0.1,
                                  height_shift_range=0.1,shear_range=0.1,zoom_range=0.1)
test_data_gen=ImageDataGenerator(rescale=1./255)
train_data=train_data_gen.flow_from_directory(train_dir,batch_size=32,
                                         color_mode='rgb',shuffle=True,class_mode='categorical',
                                         target_size=(IMG_WIDTH,IMG_HEIGHT))
test_data=train_data_gen.flow_from_directory(test_dir,batch_size=32,
                                         color_mode='rgb',shuffle=True,class_mode='categorical',
                                         target_size=(IMG_WIDTH,IMG_HEIGHT))

/content/gdrive/MyDrive/Colab Notebooks/dataset/cats_dogs_filtered/cats_and_dogs_filtered/train
Found 1999 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
print(train_data.class_indices.items())
print(test_data.class_indices.items())

dict_items([('cats', 0), ('dogs', 1)])
dict_items([('cats', 0), ('dogs', 1)])


In [ ]:
print(type(train_data))
print(type(train_data[0]))

<class 'keras.preprocessing.image.DirectoryIterator'>
<class 'tuple'>


In [ ]:
#import matplotlib.pyplot as plt
#import matplotlib.cm as cm
#plt.figure(figsize=(10,10))
#for index in range(25):
  #plt.subplot(5,5,index+1)
  #plt.imshow(train_data[index].numpy(),cmap=cm.winter)
  #plt.axis('off')
#plt.show()

In [ ]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
#/content/gdrive/MyDrive/Colab Notebooks/dataset/cats_dogs_filtered/cats_and_dogs_filtered/train/cats/cat.0.jpg
path = Path("/content/gdrive/MyDrive/Colab Notebooks/dataset/cats_dogs_filtered/cats_and_dogs_filtered/train/cats").rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)
            
path = Path("/content/gdrive/MyDrive/Colab Notebooks/dataset/cats_dogs_filtered/cats_and_dogs_filtered/train/dogs").rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(2e-5),metrics=['accuracy'])
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
save_file_name='/content/gdrive/MyDrive/Colab Notebooks/cats_and_dogs_filtered_Xception_Colab.h5'
checkpoint=ModelCheckpoint(save_file_name, monitor='val_loss',
                           verbose=1,save_best_only=True,mode='auto')
earlystopping=EarlyStopping(monitor='val_loss',patience=5)
hist=model.fit(train_data,epochs=30,validation_data=test_data,callbacks=[checkpoint,earlystopping])

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.plot(hist.history['loss'],label='train loss')
plt.plot(hist.history['val_loss'],label='validation loss')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.grid()
plt.plot(hist.history['accuracy'],label='train accuracy')
plt.plot(hist.history['val_accuracy'],label='validation accuracy')
plt.legend(loc='best')
plt.show()

In [ ]:
import os
import shutil
path_test='/content/gdrive/MyDrive/Colab Notebooks/dataset/cats_dogs_filtered/cats_and_dogs_filtered'
if os.path.exists(os.path.join(path_test,'test_image_dir/')):
  shutil.rmtree(os.path.join(path_test,'test_image_dir/'))
  print(' ... /test_image_dir/   is removed')
import zipfile
with zipfile.ZipFile(os.path.join(path_test,'test_image_dir.zip','r')) as target_file:
  target_file.extractall(os.path.join(path_test,'test_image_dir/'))




In [ ]:
import cv2
import glob
test_img_list=[]
test_img_name_list=glob.glob(os.path.join(path_test,'test_image_dir/*'))
for i in range(len(test_img_name_list)):
  src_img=cv2.imread(test_img_name_list[i],cv2.IMREAD_COLOR)
  src_img=cv2.resize(src_img,dsize=(IMG_WIDTH,IMG_HEIGHT)
  dst_img=cv2.cvtColor(src_img,cv2.COLOR_BGR2RGB)
  dst_img=dst_img/255.0
  test_img_list.append(dst_img)


SyntaxError: ignored

In [ ]:
plt.figure(figsize=(8,6))
for i in range(len(test_img_list)):
  plt.subplot(2,3,i+1)
  plot.axis('off')
  plt.imshow(test_img_list[i])
plt.show()
for i in range(len(test_img_list)):
  print(test_img_list[i].shape)

In [ ]:
pres=model.predict(np.array(test_img_list))
class_name=['cat','dog']
plt.figure(figsize=(8,6))
for i in range(len(pred))
  plt.subplot(2,3,i+1)
  predition=str(class_name[np.argmax(pred[i])])
  probility='{0:0.2f'.format(100*max(pred[i]))
  title_str=predition+','+probility+'%'
  plot.axis('off')
  plt.title(title_str)
  plt.imshow(test_img_list[i])
plt.show()